In [8]:
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tf_pose import common
import tf_slim as slim
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

In [2]:
def keypoints(human):
    scores = []
    keypoints = str(str(str(human).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)
        scores.append(float(str(keypoints[i]).split('score=')[1][0:4]))
   # human=np.array(human)

    
        
    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    return(keypts_array, scores)

In [9]:
model='mobilenet_thin'
resize='480x432'
w, h = model_wh(resize)

In [10]:
e = TfPoseEstimator(get_graph_path(model), target_size=(w, h))

[2022-04-17 18:06:31,649] [TfPoseEstimator] [INFO] loading graph from /Users/andrei-macpro/anaconda3/envs/openpose/lib/python3.7/site-packages/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=480x432)
2022-04-17 18:06:31,649 INFO loading graph from /Users/andrei-macpro/anaconda3/envs/openpose/lib/python3.7/site-packages/tf_pose_data/graph/mobilenet_thin/graph_opt.pb(default size=480x432)
2022-04-17 18:06:31.788076: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-17 18:06:31.841759: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


TfPoseEstimator/MobilenetV1/Conv2d_0/weights
TfPoseEstimator/image
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_0/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_0/Relu
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Relu
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D_bn_offset
TfPoseEstimator/M

In [11]:
video_path = '/Users/andrei-macpro/Documents/Data/excerpt.mp4'

In [12]:
cap = cv2.VideoCapture(video_path)

In [173]:
scores = []
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        frame_no+=1
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        
        humans_0 = np.array(humans[0])
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print((str(humans_0).split('score=')[1][0:4]))
        scores.append(x[1][0:4] for x in str(humans_0).split('score='))
       
        print(scores)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            score = keypoints[i][-10]
            pnt = tuple(map(float, pnt.split(', ')))
            print(type(pnt))
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])

0.65
[<generator object <genexpr> at 0x7fa8d4a588d0>]
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
[[0.64 0.28]
 [0.7  0.29]
 [0.64 0.26]
 [0.57 0.37]
 [0.54 0.46]
 [0.75 0.32]
 [0.78 0.42]
 [0.7  0.46]
 [0.63 0.5 ]
 [0.64 0.66]
 [0.64 0.73]
 [0.7  0.5 ]
 [0.7  0.66]
 [0.64 0.26]
 [0.66 0.27]
 [0.7  0.27]]
0.68
[<generator object <genexpr> at 0x7fa8d4a588d0>, <generator object <genexpr> at 0x7fa8b0c665d0>]
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
[[0.64 0.28]
 [0.69 0.29]
 [0.64 0.26]
 [0.57 0.37]
 [0.54 0.46]
 [0.75 0.32]
 [0.78 0.42]
 [0.7  0.46]
 [0.63 0.5 ]
 [0.64

KeyboardInterrupt: 

In [117]:
scores

['0', '.', '7', '3']

In [28]:
showBG = True
fps_time = 0
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print(keypoints)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            pnt = tuple(map(float, pnt.split(', ')))
            print(pnt)
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])
        print(image.shape)
        keypts_array = keypts_array.astype(int)
        print(f"keypoints for person 1 {keypts_array}")
        minimum = np.amin(keypts_array, axis=0)
        maximum = np.amax(keypts_array, axis=0)
        #rect = cv2.rectangle(image,(minimum[0]-20, minimum[1]-20),(maximum[0]+20,maximum[1]+20),(0,255,0),2)
        #cv2.boundingRect(minimum[0]-20:minimum[1]-20,maximum[0]+20:maximum[1]+20)
        #x,y,w,h = cv2.boundingRect((minimum[0]-20, minimum[1]-20,maximum[0]+20,maximum[1]+20))
        #roi=image[y:y+h,x:x+w]
        #https://stackoverflow.com/questions/54054865/given-pixel-label-draw-a-bounding-box-in-python
        #cv2.imshow('rect.png', rect)
        #image = np.zeros(image.shape)
        #cv2.imshow('image', image)
        rect = image[minimum[1]:maximum[1],minimum[0]:maximum[0]]
        cv2.imwrite('rect.png', rect)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
       # cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       # cv2.imshow('tf-pose-estimation result', image)
        #fps_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

['["[\'1", "(0.40, 0.11)', '0.61 \', \'2", "(0.34, 0.12)', '0.51 \', \'3", "(0.32, 0.24)', '0.71 \', \'4", "(0.37, 0.32)', '0.74 \', \'5", "(0.45, 0.10)', '0.50 \', \'6", "(0.49, 0.22)', '0.71 \', \'7", "(0.43, 0.30)', '0.52 \', \'8", "(0.38, 0.32)', '0.34 \', \'9", "(0.39, 0.42)', '0.13 \', \'11", "(0.45, 0.31)', '0.43 \', \'12", "(0.44, 0.43)', '0.21\']"]']
(0.4, 0.11)
(0.34, 0.12)
(0.32, 0.24)
(0.37, 0.32)
(0.45, 0.1)
(0.49, 0.22)
(0.43, 0.3)
(0.38, 0.32)
(0.39, 0.42)
(0.45, 0.31)
(0.44, 0.43)
[[0.4  0.11]
 [0.34 0.12]
 [0.32 0.24]
 [0.37 0.32]
 [0.45 0.1 ]
 [0.49 0.22]
 [0.43 0.3 ]
 [0.38 0.32]
 [0.39 0.42]
 [0.45 0.31]
 [0.44 0.43]]
(576, 768, 3)
keypoints for person 1 [[307  63]
 [261  69]
 [245 138]
 [284 184]
 [345  57]
 [376 126]
 [330 172]
 [291 184]
 [299 241]
 [345 178]
 [337 247]]
['["[\'1", "(0.40, 0.11)', '0.61 \', \'2", "(0.34, 0.12)', '0.52 \', \'3", "(0.32, 0.24)', '0.68 \', \'4", "(0.36, 0.32)', '0.71 \', \'5", "(0.45, 0.10)', '0.51 \', \'6", "(0.49, 0.22)', '0.72 \'

KeyboardInterrupt: 

In [66]:
scores

['o', '.', '.', '.', '.', '.', '.']

In [43]:
(0.44, 0.12) *(image.shape[1],image.shape[0])

TypeError: can't multiply sequence by non-int of type 'tuple'

In [14]:
from operator import mul
res = tuple(map(mul, (0.44, 0.12), (image.shape[1],image.shape[0])))

In [15]:
tuple(map(float, res))

(337.92, 69.12)

In [16]:
type(res)

tuple

In [17]:
res2 = np.array(res)


In [18]:
res2.astype(int)

array([337,  69])

In [63]:
def keypoints(human):
    scores = []
    keypoints = str(str(str(human).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    for i in range (len(keypoints)-1): 
        pnt = keypoints[i][-11:-1]
        pnt = tuple(map(float, pnt.split(', ')))
        keypoints_list.append(pnt)
    human=np.array(human)
    for i in str(human).split(" score="):
        if i[0:4]=="Body":
            continue
        else:
            scores.append(float(i[0:4]))
    scores=np.array(scores)
    keypts_array = np.array(keypoints_list)
    keypts_array = keypts_array*(image.shape[1],image.shape[0])
    keypts_array = keypts_array.astype(int)
    return(keypts_array, scores)

In [13]:
cap = cv2.VideoCapture(video_path)
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    
    keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
    keypoints_list=[]
    humans = np.array(humans[0])
    print(humans)
    for i in str(humans).split(" score="):
        if i[0:4]=="Body":
            continue
        else:
            print(i[0:4])

BodyPart:0-(0.65, 0.25) score=0.58 BodyPart:1-(0.68, 0.27) score=0.62 BodyPart:2-(0.62, 0.26) score=0.56 BodyPart:3-(0.60, 0.33) score=0.40 BodyPart:4-(0.59, 0.41) score=0.65 BodyPart:5-(0.74, 0.29) score=0.49 BodyPart:6-(0.77, 0.40) score=0.72 BodyPart:7-(0.70, 0.44) score=0.74 BodyPart:8-(0.63, 0.47) score=0.49 BodyPart:9-(0.63, 0.58) score=0.43 BodyPart:10-(0.65, 0.71) score=0.33 BodyPart:11-(0.70, 0.47) score=0.47 BodyPart:12-(0.68, 0.58) score=0.26 BodyPart:13-(0.66, 0.72) score=0.10 BodyPart:14-(0.65, 0.23) score=0.71 BodyPart:15-(0.67, 0.24) score=0.66 BodyPart:17-(0.71, 0.25) score=0.54
0.58
0.62
0.56
0.40
0.65
0.49
0.72
0.74
0.49
0.43
0.33
0.47
0.26
0.10
0.71
0.66
0.54
BodyPart:0-(0.66, 0.25) score=0.60 BodyPart:1-(0.68, 0.27) score=0.63 BodyPart:2-(0.62, 0.26) score=0.59 BodyPart:3-(0.60, 0.32) score=0.59 BodyPart:4-(0.59, 0.41) score=0.72 BodyPart:5-(0.74, 0.29) score=0.52 BodyPart:6-(0.77, 0.40) score=0.76 BodyPart:7-(0.69, 0.45) score=0.68 BodyPart:8-(0.63, 0.45) score=0.5

KeyboardInterrupt: 

In [65]:
cap = cv2.VideoCapture(video_path)
human_no = []
frame_no = 0
failed_frames = []
human_1 = []
human_2 = []
human_3 =[]
probs_1 = []
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    frame_no+=1
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    human_no.append(len(humans))
    
    if len(humans)>2:
        keypoints_3 = keypoints(humans[2])
        keypoints_2 = keypoints(humans[1])
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    
    elif len(humans)>1:
        keypoints_2 = keypoints(humans[1])
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    elif len(humans) == 1:
        keypoints_1 = keypoints(humans[0])
        failed_frames.append('success')
    else:
        failed_frames.append(frame_no)
    human_1.append({frame_no:keypoints_1})
    human_2.append({frame_no:keypoints_2})
    human_3.append({frame_no:keypoints_3})
 

    
 
cap.release()
human_1 = np.array(human_1)
human_2 = np.array(human_2)
human_3 = np.array(human_3)
human_no = np.array(human_no)
failed_frames = np.array(failed_frames) 

In [66]:
failed_frames

array(['success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 'success', 'success',
       'success', 'success', 'success', 'success', 

In [71]:
human_1[0]

{1: (array([[499, 144],
         [522, 155],
         [476, 149],
         [460, 190],
         [453, 236],
         [568, 167],
         [591, 230],
         [537, 253],
         [483, 270],
         [483, 334],
         [499, 408],
         [537, 270],
         [522, 334],
         [506, 414],
         [499, 132],
         [514, 138],
         [545, 144]]),
  array([0.58, 0.62, 0.56, 0.4 , 0.65, 0.49, 0.72, 0.74, 0.49, 0.43, 0.33,
         0.47, 0.26, 0.1 , 0.71, 0.66, 0.54]))}

In [75]:
for i in range(18):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [14]:

cap = cv2.VideoCapture(video_path)
while(cap.isOpened()):
    ret_val, image = cap.read()
    if ret_val == False:
        break
    humans = e.inference(image,
                         resize_to_default=(w > 0 and h > 0),
                         upsample_size=4.0)
    human_0 = humans[0]

KeyboardInterrupt: 

In [15]:
human_0

BodyPart:0-(0.66, 0.27) score=0.62 BodyPart:1-(0.68, 0.27) score=0.55 BodyPart:2-(0.62, 0.26) score=0.57 BodyPart:3-(0.60, 0.32) score=0.54 BodyPart:4-(0.59, 0.40) score=0.68 BodyPart:5-(0.73, 0.30) score=0.64 BodyPart:6-(0.74, 0.43) score=0.67 BodyPart:7-(0.66, 0.49) score=0.66 BodyPart:8-(0.63, 0.47) score=0.36 BodyPart:9-(0.63, 0.60) score=0.17 BodyPart:10-(0.65, 0.74) score=0.15 BodyPart:11-(0.70, 0.49) score=0.34 BodyPart:12-(0.69, 0.60) score=0.08 BodyPart:13-(0.66, 0.73) score=0.10 BodyPart:14-(0.65, 0.25) score=0.71 BodyPart:15-(0.68, 0.25) score=0.63 BodyPart:16-(0.64, 0.23) score=0.16 BodyPart:17-(0.70, 0.25) score=0.48

In [10]:
(str(human_0).split("-"))

['BodyPart:0',
 '(0.65, 0.27) score=0.66 BodyPart:1',
 '(0.68, 0.28) score=0.58 BodyPart:2',
 '(0.62, 0.27) score=0.57 BodyPart:3',
 '(0.60, 0.32) score=0.53 BodyPart:4',
 '(0.59, 0.40) score=0.70 BodyPart:5',
 '(0.73, 0.30) score=0.62 BodyPart:6',
 '(0.73, 0.43) score=0.61 BodyPart:7',
 '(0.65, 0.51) score=0.58 BodyPart:8',
 '(0.64, 0.44) score=0.36 BodyPart:9',
 '(0.63, 0.62) score=0.16 BodyPart:10',
 '(0.65, 0.75) score=0.14 BodyPart:11',
 '(0.70, 0.50) score=0.41 BodyPart:12',
 '(0.68, 0.62) score=0.06 BodyPart:14',
 '(0.65, 0.25) score=0.61 BodyPart:15',
 '(0.67, 0.26) score=0.71 BodyPart:17',
 '(0.70, 0.25) score=0.50']

In [28]:
showBG = True
fps_time = 0
if cap.isOpened():
    while True:
        ret_val, image = cap.read()
        humans = e.inference(image,
                             resize_to_default=(w > 0 and h > 0),
                             upsample_size=4.0)
        keypoints = str(str(str(humans[0]).split('BodyPart:')[1:]).split('-')).split(' score=')
        print(keypoints)
        keypoints_list=[]
        for i in range (len(keypoints)-1): 
            pnt = keypoints[i][-11:-1]
            pnt = tuple(map(float, pnt.split(', ')))
            print(pnt)
            keypoints_list.append(pnt)
        keypts_array = np.array(keypoints_list)
        print(keypts_array)
        keypts_array = keypts_array*(image.shape[1],image.shape[0])
        print(image.shape)
        keypts_array = keypts_array.astype(int)
        print(f"keypoints for person 1 {keypts_array}")
        minimum = np.amin(keypts_array, axis=0)
        maximum = np.amax(keypts_array, axis=0)
        #rect = cv2.rectangle(image,(minimum[0]-20, minimum[1]-20),(maximum[0]+20,maximum[1]+20),(0,255,0),2)
        #cv2.boundingRect(minimum[0]-20:minimum[1]-20,maximum[0]+20:maximum[1]+20)
        #x,y,w,h = cv2.boundingRect((minimum[0]-20, minimum[1]-20,maximum[0]+20,maximum[1]+20))
        #roi=image[y:y+h,x:x+w]
        #https://stackoverflow.com/questions/54054865/given-pixel-label-draw-a-bounding-box-in-python
        #cv2.imshow('rect.png', rect)
        #image = np.zeros(image.shape)
        #cv2.imshow('image', image)
        rect = image[minimum[1]:maximum[1],minimum[0]:maximum[0]]
        cv2.imwrite('rect.png', rect)
        image = TfPoseEstimator.draw_humans(image, humans, imgcopy=True)
       # cv2.putText(image, "FPS: %f" % (1.0 / (time.time() - fps_time)), (10, 10),
        #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       # cv2.imshow('tf-pose-estimation result', image)
        #fps_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

['["[\'1", "(0.40, 0.11)', '0.61 \', \'2", "(0.34, 0.12)', '0.51 \', \'3", "(0.32, 0.24)', '0.71 \', \'4", "(0.37, 0.32)', '0.74 \', \'5", "(0.45, 0.10)', '0.50 \', \'6", "(0.49, 0.22)', '0.71 \', \'7", "(0.43, 0.30)', '0.52 \', \'8", "(0.38, 0.32)', '0.34 \', \'9", "(0.39, 0.42)', '0.13 \', \'11", "(0.45, 0.31)', '0.43 \', \'12", "(0.44, 0.43)', '0.21\']"]']
(0.4, 0.11)
(0.34, 0.12)
(0.32, 0.24)
(0.37, 0.32)
(0.45, 0.1)
(0.49, 0.22)
(0.43, 0.3)
(0.38, 0.32)
(0.39, 0.42)
(0.45, 0.31)
(0.44, 0.43)
[[0.4  0.11]
 [0.34 0.12]
 [0.32 0.24]
 [0.37 0.32]
 [0.45 0.1 ]
 [0.49 0.22]
 [0.43 0.3 ]
 [0.38 0.32]
 [0.39 0.42]
 [0.45 0.31]
 [0.44 0.43]]
(576, 768, 3)
keypoints for person 1 [[307  63]
 [261  69]
 [245 138]
 [284 184]
 [345  57]
 [376 126]
 [330 172]
 [291 184]
 [299 241]
 [345 178]
 [337 247]]
['["[\'1", "(0.40, 0.11)', '0.61 \', \'2", "(0.34, 0.12)', '0.52 \', \'3", "(0.32, 0.24)', '0.68 \', \'4", "(0.36, 0.32)', '0.71 \', \'5", "(0.45, 0.10)', '0.51 \', \'6", "(0.49, 0.22)', '0.72 \'

KeyboardInterrupt: 

In [16]:


subscriptable = str(human_0).split("BodyPart:")[1:]


In [24]:
kpts = []
coords_x = []
coords_y = []
probs = []
all_points = [x for x in range(18)]
big_dic = []
subscriptable = str(human_0).split("BodyPart:")[1:]
for x in subscriptable:
    kpts.append(int((x.split('-')[0])))
    coords_x.append(float((str(x.split('-')[1]).split(" score=")[0][1:5]))*image.shape[1])
    coords_y.append(float((str(x.split('-')[1]).split(" score=")[0][6:11]))*image.shape[0])
    probs.append(float(str(x.split('-')[1]).split(" score=")[1]))
for x,y,cox, coy, prob in zip(all_points,kpts, coords_x, coords_y, probs):
    if x not in kpts:
        big_dic.append({x:['no_x','no_y','no_prob']})
    else:
        big_dic.append({y:[round(cox,2),round(coy,2), prob]})

#return(np.array(kpts), np.array(coords_x), 

In [176]:
huh = str(human_0).replace('BodyPart:', "").replace("score=", "")
huh

'0-(0.65, 0.27) 0.66 1-(0.68, 0.28) 0.58 2-(0.62, 0.27) 0.57 3-(0.60, 0.32) 0.53 4-(0.59, 0.40) 0.70 5-(0.73, 0.30) 0.62 6-(0.73, 0.43) 0.61 7-(0.65, 0.51) 0.58 8-(0.64, 0.44) 0.36 9-(0.63, 0.62) 0.16 10-(0.65, 0.75) 0.14 11-(0.70, 0.50) 0.41 12-(0.68, 0.62) 0.06 14-(0.65, 0.25) 0.61 15-(0.67, 0.26) 0.71 17-(0.70, 0.25) 0.50'

In [25]:
big_dic

[{0: [506.88, 155.52, 0.62]},
 {1: [522.24, 155.52, 0.55]},
 {2: [476.16, 149.76, 0.57]},
 {3: [460.8, 184.32, 0.54]},
 {4: [453.12, 230.4, 0.68]},
 {5: [560.64, 172.8, 0.64]},
 {6: [568.32, 247.68, 0.67]},
 {7: [506.88, 282.24, 0.66]},
 {8: [483.84, 270.72, 0.36]},
 {9: [483.84, 345.6, 0.17]},
 {10: [499.2, 426.24, 0.15]},
 {11: [537.6, 282.24, 0.34]},
 {12: [529.92, 345.6, 0.08]},
 {13: [506.88, 420.48, 0.1]},
 {14: [499.2, 144.0, 0.71]},
 {15: [522.24, 144.0, 0.63]},
 {16: [491.52, 132.48, 0.16]},
 {17: [537.6, 144.0, 0.48]}]

In [149]:
keypoint = []
for i in range(len(keyp_coord)):
    keypoint.append(int(keyp_coord[i][2]))

In [155]:
str(human_0).find('score=')

24

In [182]:
import json
json.loads(human_0)

TypeError: the JSON object must be str, bytes or bytearray, not ndarray